### Info

Script to download Level3A (WASP-generated) Sentinel2A data processed by the German Aerospace Centre.
This data contains low-cloud/cloudfree, monthly averages of Sentinel2A scenes.

Processing steps:
* download tiles based on specified aoi, timeframe & bands
* clipping tiles to aoi extent & creating a mosaic for each timestep and band

What is not covered so far & may be improved in upcoming releases
* consideration of quality control mask provided along with data for two reasons
  + inaccuracy of the mask
  + low significance of mask for clouds as processing chain removes almost all cloudy pixels

But mask can be downloaded & clipped for AoI and then statistics be calculated to decide if it should be excluded (if above threshold value).
Also visual inspection possible directly in script via plotting rbg image for region. Print statistics for each layer (on proprotion of masked pixels). Further idea: Script for downloading L2A data for specific AoI with cloud thresholding possibility (first look what is already provided on github in this regard). 

In [1]:
# import libraries
import geopandas as gpd
import glob
import minihit
import numpy as np
import os
import rasterio
import requests
import shutil
import wget
import zipfile

from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
from tqdm.notebook import tqdm
from rasterio.mask import mask
from rasterio.merge import merge

### Test data ###
For testing purposes data for three different AoIs with different field structures & varying S2 imagery periods was downloaded 
* upper rhine area: ~120km2, great variety of field sizes & mixed use, Apr-Sep, cloudfree
* coastal landscape: ~25km2, mostly meadows & partly delineated by bushes, June-Oct, cloudfree
* magdeburger boerde: ~85km2, exclusively arable farming with large field sizes, Mar-Oct, partly cloudy

In [21]:
### variables to set ###
# download dir specifies existing folder to write scences to disk
download_dir = os.path.join(os.getcwd(), "test_data", "magdeburger_boerde", "s2_l3a_data")

# aoi has to be specified as geojson
# must be in parent dir of s2_l3_data folder
# can be created online, e.g. via https://geojson.io
aoi_geojson = os.path.join(os.path.dirname(download_dir), "aoi.geojson")

# timeframe
years = [2021]
months = [x for x in range(3,11)]

# bands
bands = ["B2", "B3", "B4", "B8"]

In [15]:
# create dataset with utm tiles
# download utm tile shp
url = "https://codeload.github.com/justinelliotmeyers/Sentinel-2-Shapefile-Index/zip/refs/heads/master"
wget.download(url, download_dir)
with zipfile.ZipFile(os.path.join(download_dir, "Sentinel-2-Shapefile-Index-master.zip"), "r") as zip_ref:
    zip_ref.extractall(download_dir)

# read utm & delete copies on disk
utm = gpd.read_file(os.path.join(download_dir, "Sentinel-2-Shapefile-Index-master", "sentinel_2_index_shapefile.shp"))
shutil.rmtree(os.path.join(download_dir, "Sentinel-2-Shapefile-Index-master"))
os.remove(os.path.join(download_dir, "Sentinel-2-Shapefile-Index-master.zip"))

# restrict utm dataset to germany
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
germany = world[world["name"] == "Germany"]

# split utm tiles in overlapping areas
_utm = utm.sjoin(germany).iloc[:,0:2].overlay(utm.sjoin(germany).iloc[:,0:2], how='union')
_utm = _utm[_utm["Name_1"] != _utm["Name_2"]].dropna()

_utm2 = utm.sjoin(germany).iloc[:,0:2].overlay(_utm, how='symmetric_difference')
_utm2 = _utm2.rename(columns={"Name":"Name_3"})
utm = _utm.append(_utm2)

C:\Users\felix\AppData\Roaming\Python\Python310\site-packages\geopandas\geodataframe.py:2196: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(
C:\Users\felix\AppData\Roaming\Python\Python310\site-packages\geopandas\geodataframe.py:2196: UserWarning: `keep_geom_type=True` in overlay resulted in 48 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [16]:
# intersect aoi with utm to determine relevant utm tiles 
aoi = gpd.read_file(aoi_geojson)
intersect = gpd.sjoin(aoi, utm)
set_utm_tiles = [{x,y,z} for x,y,z in list(zip(intersect["Name_1"], intersect["Name_2"], intersect["Name_3"]))]
set_utm_tiles = [{x for x in tile if x==x} for tile in set_utm_tiles]

# solve hitting set problem (a.k.a. set covering problem)
# aim: filter absolute minimum of necessary utm tiles to cover aoi
rc = minihit.RcTree(set_utm_tiles)
rc.solve(prune=True, sort=False)
utm_tiles = sorted(list(rc.generate_minimal_hitting_sets())[0])
utm_tiles

['32UPC']

In [17]:
# construction of download urls following the schema baseurl/utm_tile/year/month/band 
baseurl = "https://download.geoservice.dlr.de/S2_L3A_WASP/files/"
download_urls = []

for tile in tqdm(utm_tiles, desc='constructing download urls & checking url validity'):
    # construct tiles
    tile_url = os.path.join(baseurl, tile[0:2], tile[2:3], tile[3:]).replace('\\','/')
    if requests.get(tile_url).status_code == 200:
        # construct years
        for year in years:
            year_url = os.path.join(tile_url, str(year)).replace('\\','/')
            if requests.get(year_url).status_code == 200:
                # construct months
                for month in months:
                    month = "0" + str(month) if len(str(month))==1 else str(month)
                    path_spec_month = "SENTINEL2X_" + str(year) + month + "15-000000-000_L3A_T" + tile + "_C_V1-2"
                    month_url = os.path.join(year_url, path_spec_month).replace('\\','/')
                    if requests.get(month_url).status_code == 200:
                        # construct bands
                        for band in bands:
                            page_bands = requests.get(month_url)
                            page_bands = BeautifulSoup(page_bands.content, "html.parser")
                            path_spec_band = path_spec_month + "_FRC_" + band + ".tif"
                            if page_bands.find_all("a", href=lambda x: x.startswith(path_spec_band)):
                                band_url = os.path.join(month_url, path_spec_band).replace('\\','/')
                                download_urls.append(band_url)
                            else:
                                print(f"URL {band_url} does not exist or cannot be queried")
                    else:
                        print(f"URL {month_url} does not exist or cannot be queried")
            else:
                print(f"URL {year_url} does not exist or cannot be queried")
    else:
        print(f"URL {tile_url} does not exist or cannot be queried")

# evaluation of the availability and memory consumption for the selected tiles
n_tiles = len(utm_tiles) * len(years) * len(months) * len(bands)

memory_bands = []
memory_bands.extend([240 for x in bands if x=='B8'])
memory_bands.extend([200 for x in bands if x in ['B2','B3','B4']])
memory_bands.extend([55 for x in bands if x in ['B5','B6','B7', 'B8A', 'B11', 'B12']])

memory_all = n_tiles * sum(memory_bands) / len(bands)

print("--- Number/Availability of Scenes ---")
print(f"The current specification of aoi, timeframe and bands corresponds to a total of {n_tiles} individual tiles to be downloaded.")
print(f"Out of this {len(download_urls)} tiles ({len(download_urls)/n_tiles:.0%}) are available on the server.")
print("")

print("--- Download volume ---")
print(f"Estimated download volume: ~{round(memory_all/1000) if memory_all>2500 else round(memory_all/1000,1)}GB")

constructing download urls & checking url validity:   0%|          | 0/1 [00:00<?, ?it/s]

--- Number/Availability of Scenes ---
The current specification of aoi, timeframe and bands corresponds to a total of 32 individual tiles to be downloaded.
Out of this 32 tiles (100%) are available on the server.

--- Download volume ---
Estimated download volume: ~7GB


In [18]:
# download of data
# using multithreading to accelerate download
# note: p.starmap stops download after each submitted chunk of tasks
# note: using p.imap works & progress bar (tqdm) can still be displayed
def download_scenes(url, dir):
    import os
    import sys
    import wget
    if not os.path.isfile(os.path.join(dir, url.split("/")[-1])):
        try:
            wget.download(url, dir)
        except:
            pass

with Pool(min(os.cpu_count(), 4)) as p:
    _ = list(tqdm(p.imap(download_scenes, download_urls, [download_dir for x in range(len(download_urls))]), 
                  total=len(download_urls),
                  desc='downloading data', 
                  smoothing=0.2))

# in case of issues: use single core download
# for url in tqdm(download_urls, desc='downloading data', smoothing=0.2):
#     if not os.path.isfile(os.path.join(download_dir, url.split("/")[-1])):
#         wget.download(url, download_dir)

print(f"A total of {len(glob.glob(os.path.join(download_dir, 'SENTINEL2X*.tif')))} files is now present in specified download directory.")

downloading data:   0%|          | 0/32 [00:00<?, ?it/s]

A total of 32 files is now present in specified download directory.


In [19]:
# procedure to mosaic and mask tiles per date & band
downloaded_tiles = glob.glob(os.path.join(download_dir, 'SENTINEL2X*.tif'))
tiles_per_date_band = ["_".join(x.split("\\")[-1].split("_")[:3]) + "*" + x.split("\\")[-1].split("_")[-1] for x in downloaded_tiles]
tiles_per_date_band = list(set(tiles_per_date_band))

for tiles in tqdm(tiles_per_date_band, desc="merging & masking of bands"):

    # open tiles
    src_files_to_mosaic = []
    for tile in glob.glob(os.path.join(download_dir, tiles)):
        src = rasterio.open(tile)
        src_files_to_mosaic.append(src)

    # merge tiles
    mosaic, out_trans = merge(src_files_to_mosaic)

    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans}
                    )

    out_dir = os.path.join(download_dir, tiles.replace("-000000-000", "").replace("*", "_"))
    with rasterio.open(out_dir, "w", **out_meta) as dest:
        dest.write(mosaic)

    # mask tiles with aoi geojson
    with rasterio.open(out_dir) as src:
        masked_mosaic, out_trans2 = mask(src, aoi["geometry"].to_crs(out_meta["crs"].to_string()), crop=True)

        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                        "height": masked_mosaic.shape[1],
                        "width": masked_mosaic.shape[2],
                        "transform": out_trans2}
                        )

    with rasterio.open(out_dir, "w", **out_meta) as dest:
        dest.write(masked_mosaic)

    #remove original data & keep only mosaic
    for rasters in src_files_to_mosaic:
        rasters.close()
        
    for tile in glob.glob(os.path.join(download_dir, tiles)):
        os.remove(tile)

merging & masking of bands:   0%|          | 0/32 [00:00<?, ?it/s]

In [20]:
# compatibility with ecognition workflow
# rename scenes
downloaded_scenes = [x for x in glob.glob(os.path.join(download_dir, 'SENTINEL2X*.tif'))]

for scene_path in downloaded_scenes:
    scene = scene_path.split("\\")[-1]
    scene_date = scene.split("_")[1]
    scene_band = scene.split("_")[-1].split(".tif")[0]
    bandidx_to_name = {"B2": "blue", "B3": "green", "B4": "red", "B8": "nir"}
    new_scene_name = f"s2_l3a_{scene_date}_{bandidx_to_name[scene_band]}.tif"
    os.rename(scene_path, os.path.join(download_dir, new_scene_name))


In [19]:
# output buffered aoi
# aim: avoid border effects
# means: 20m inwards buffer via Lambert Azimuthal Equal Area projection 
# aoi_ecognition = aoi.to_crs(crs=9821).buffer(-2000).to_crs(crs=4326)
# aoi_ecognition_path = os.path.join(os.path.dirname(download_dir), "aoi_ecognition.geojson")
# aoi_ecognition.to_file(aoi_ecognition_path, driver='GeoJSON')